# RAG Architecture Sample in Python (Azure AI Search)
여기서는 아래 이미지와 같은 RAG 아키텍처를 만들어 볼 것이다.

<img src="./images/02_001.png" width="70%">


# 사전 준비
이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:
- Azure AI Search 리소스의 엔드포인트 및 쿼리 API 키
- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 `text-embedding-ada-002` Embeddings 모델. 이 모델의 API 버전은 `2023-05-15`을 사용했다. 배포 이름은 모델 이름과 동일하게 `text-embedding-ada-002`로 명명했다.
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI Embeddings 모델의 배포 이름
  - OpenAI API 버전
- Python (이 예제는 버전 3.10.x로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.

## 패키지 설치

In [ ]:
!pip install azure-search-documents==11.4.0
!pip install openai

In [ ]:
import azure.search.documents
azure.search.documents.__version__

## 라이브러리 및 환경변수 불러오기

In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)

## Azure AI Search 연동 설정

In [ ]:
service_endpoint: str = "<Your search service endpoint>"
service_query_key: str = "<Your search service query key>"
index_name: str = "gptkbindex" # 자동 구축시 기본으로 설정

credential = AzureKeyCredential(service_query_key)

## Azure OpenAI 설정

In [ ]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "chat" # 자동 구축시 기본으로 설정
AZURE_OPENAI_EMB_DEPLOYMENT="embedding" # 자동 구축시 기본으로 설정

In [ ]:
from openai import AzureOpenAI
from openai.types.chat import (
    ChatCompletion,
    ChatCompletionChunk,
)

from tenacity import retry, wait_random_exponential, stop_after_attempt  

openai_client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2024-02-01",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# title 필드와 content 필드의 Embeddings를 생성하는 함수
def generate_embeddings(text, model=AZURE_OPENAI_EMB_DEPLOYMENT):
    return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

# 1. 검색 쿼리 생성
최신 질문과 채팅 이력을 기반으로 프롬프트를 작성하며 GPT-3.5 Turbo 모델을 사용하여 검색 쿼리를 생성한다. 검색 쿼리의 형식을 맞추기 위해 Few-shot 예제를 준비하여 정확도를 높였다.

## 1.1. 시스템 메시지 설정

In [ ]:
# Query generation prompt
query_prompt_template = """
다음은 과거 대화 이력과 한국사 지식을 기반으로 질문과 답변을 주고받는 중에 사용자가 새로운 질문을 한 상황이야.
과거 대화 이력과 새로운 질문에 기반해서 검색 쿼리를 생성해줘.
검색 쿼리에는 인용한 파일이나 문서 이름(예: info.txt 또는 doc.pdf)을 포함해줘.
검색 쿼리에는 괄호([] 또는 <<>>) 안에 있는 문자를 포함시켜선 안돼.
검색 쿼리를 생성할 수 없으면 숫자 0이라고만 답변해줘.
"""

messages = [{'role': 'system', 'content': query_prompt_template}]

## 1.2. Few-shot 예제 설정

In [ ]:
# Few-shot Samples
query_prompt_few_shots = [
    {'role' : 'user', 'content' : '이순신은 어떤 인물이야?' },
    {'role' : 'assistant', 'content' : '이순신 인물 역사' },
    {'role' : 'user', 'content' : '이순신의 공적에 대해서 알려줘.' },
    {'role' : 'assistant', 'content' : '이순신 인물 공적' }
]

for shot in query_prompt_few_shots:
    messages.append({'role': shot.get('role'), 'content': shot.get('content')})

## 1.3. 사용자의 질문

In [ ]:
# User query
user_q = "최충헌은 어떤 인물이야?"
messages.append({'role': 'user', 'content': user_q})

## 1.4. 전송할 메시지 확인

In [ ]:
messages

## 1.5. 검색 쿼리 생성

In [ ]:
chat_completion: ChatCompletion = openai_client.chat.completions.create(
    messages=messages,
    model=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    temperature=0.0,
    max_tokens=100,
    n=1)

query_text = chat_completion.choices[0].message.content
print(query_text)

# 2. 검색 인덱스에서 연관 문서 취득(Retrieve)
1. 에서 생성한 검색 쿼리를 사용하여 Azure AI Search로 검색을 수행한다. 이 예제에서는 검색 쿼리와 벡터를 조합한 하이브리드 검색을 수행한다.

In [ ]:
def nonewlines(s: str) -> str:
    return s.replace('\n', ' ').replace('\r', ' ').replace('[', '【').replace(']', '】')

In [ ]:
search_client = SearchClient(service_endpoint, index_name, credential=credential)
docs = search_client.search(
    search_text=query_text,
    filter=None,
    top=3,
    vector_queries=[VectorizedQuery(vector=generate_embeddings(query_text), k_nearest_neighbors=3, fields="embedding")]
)

In [ ]:
results =[" SOURCE:" + doc['sourcepage'] + ": " + nonewlines(doc['content']) for doc in docs]
print(results)

# 3. ChatGPT를 활용한 응답 생성

Azure AI Search의 검색 결과나 채팅 이력을 활용해서 콘텍스트나 내용에 알맞은 응답을 생성한다. 이 때, 프롬프트를 활용해서 출처를 출력하도록 지시한다. 출처에는 Azure AI Search의 파일 이름이라는 필드의 값을 사용한다.

시스템 메시지의 정확도를 높이기 위해 일부는 영어로 기술한다.

In [ ]:
# System message
system_message_chat_conversation = """
너는 한국의 무신정권 역사에 관한 문제를 답변해주는 역사 교수야. 
If you cannot guess the answer to a question from the SOURCE, answer "I don't know".
Answers must be in Korean.

# Restrictions
- The SOURCE prefix has a colon and actual information after the filename, and each fact used in the response must include the name of the source.
- To reference a source, use a square bracket. For example, [info1.txt]. Do not combine sources, but list each source separately. For example, [info1.txt][info2.pdf].
"""

messages = [{'role': 'system', 'content': system_message_chat_conversation}]

## 3.1. 콘텍스트 확장(Augument)

In [ ]:
# User query
user_q = "최충헌은 어떤 인물이야?"
# Context from Azure AI Search
context = "\n".join(results)
messages.append({'role': 'user', 'content': user_q + "\n\n" + context}) 

## 3.2. 전송할 메시지 확인

In [ ]:
messages

## 3.3. 응답 생성(Generation)

In [ ]:
# ChatCompletion으로 응답 생성하기
chat_coroutine = openai_client.chat.completions.create(
    model=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    messages=messages,
    temperature=0.0,
    max_tokens=1024,
    n=1,
    stream=False
)

print(chat_coroutine.choices[0].message.content)